In [1]:
#importing dependencies

import re
from youtube_transcript_api import YouTubeTranscriptApi
import torch
import torchaudio
import openai
import textwrap
import os
from openai import OpenAI


In [2]:
# Specify the YouTube video URL
youtube_url = "https://www.youtube.com/" # Add the URL of the YouTube video here

# Extract the video ID from the URL using regular expressions
match = re.search(r"v=([A-Za-z0-9_-]+)", youtube_url)
if match:
    video_id = match.group(1)
else:
    raise ValueError("Invalid YouTube URL")

# Get the transcript from YouTube
transcript = YouTubeTranscriptApi.get_transcript(video_id)

# Concatenate the transcript into a single string
transcript_text = ""
for segment in transcript:
    transcript_text += segment["text"] + " "
print(transcript_text)


hi there this is Valentine and welcome to another tutorial this time I wanted to introduce you to get lab CI soget lab continuous integration tool this is built in the get lab tool you do not need to download any software or anything like that all you need is a browser and get lab account it's totally free so if you haven't created an account do that now and once you are logged into your account you should see something like new project here on the right so we're gonna create a very simple project and we are gonna create a continuous integration pipeline it will be very very simple but just to demonstrate the basic functionality of get lab so go ahead click new project and this will be something like my simple pipeline and right here below you have the possibility of selecting the visibility you can make it public if you want but I will leave it private because I'm just playing around so right here below I'm gonna create the project now once that is ready you will have normal repositor

In [3]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "Helsinki-NLP/opus-mt-zh-en"
translator = pipeline("translation", model=model_checkpoint)

# Define the maximum sequence length
max_length = 512

# Split the input text into smaller segments
segments = [transcript_text[i:i+max_length] for i in range(0, len(transcript_text), max_length)]

# Translate each segment and concatenate the results
translated_text = ""
for segment in segments:
    result = translator(segment)
    translated_text += result[0]['translation_text']

print(translated_text)


C:\Users\craig\Python\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


I wanted to introduce you to get lab cl so now and once you are logged into your account you should see something like new project here on the right we're gonna sell a very small project and we're on the other side.If you want to make it public if you want but I will leave it prevate because I'm just playing around so right here- See I got mo so this is how the file should be named and this is a condition it will easily reflect it so we're trying to do here is will be a built stage and there will be a test stage now in the building stage I will do something very small I create a file and then in the head stage I will test that file was actually laid in the original stageI'm gonna say something like stickies and then with the tap I'm gonna say there's a built stage and there is a testage so these are the two pages that we'll have and now we have to deal with the jobs themselves so well can I have a built and you can declare it as you can name it as you want but it's important now to say

In [ ]:
from transformers import pipeline, AutoTokenizer

# Instantiate the tokenizer and the summarization pipeline
tokenizer = AutoTokenizer.from_pretrained('stevhliu/my_awesome_billsum_model')
summarizer = pipeline("summarization", model='stevhliu/my_awesome_billsum_model', tokenizer=tokenizer)

# Define chunk size in number of words
chunk_size = 200 # you may need to adjust this value depending on the average length of your words

# Split the text into chunks
words = transcript_text.split()
chunks = [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Summarize each chunk
summaries = []
for chunk in chunks:
    # Summarize the chunk
    summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)

    # Extract the summary text
    summary_text = summary[0]['summary_text']

    # Add the summary to our list of summaries
    summaries.append(summary_text)

# Join the summaries back together into a single summary
final_summary = ' '.join(summaries)

print(final_summary)


In [ ]:
def split_text_into_chunks(text, max_chunk_size):
    return textwrap.wrap(text, max_chunk_size)

client = OpenAI(api_key =os.environ['..\\.venv\\secrets\\OPENAI_API_KEY'])
#openai.api_key = "OPENAI_API_KEY" removed per https://github.com/openai/openai-python/discussions/742
max_chunk_size = 4000

transcript_chunks = split_text_into_chunks(transcript_text, max_chunk_size)
summaries = ""

for chunk in transcript_chunks:
    #Remove response = openai.ChatCompletion.create per https://github.com/openai/openai-python/discussions/742
    response = client.completions.create(
        model="gpt-3.5-turbo-16k",
        prompt="You are a helpful assistant.\n\nUser: " + chunk + "\n\nCreate short concise summary:",
        #messages=[
            #{"role": "system", "content": "You are a helpful assistant."},
            #{"role": "user", "content": f"{chunk}\n\nCreate short concise summary"}
        #],
        max_tokens=250,
        temperature=0.5
    )

    summaries += response['choices'][0]['message']['content'].strip() + " "

print("Summary:")
print(summaries)


In [ ]:
response = openai.ChatCompletion.create(
model="gpt-3.5-turbo-16k",
messages=[
{"role": "system", "content": "You are a technical instructor."},
{"role": "user", "content": transcript_text},
{"role": "user", "content": "Generate steps to follow from text."},
]
)

# The assistant's reply
guide= response['choices'][0]['message']['content']

print("Steps:")
print(guide)


In [ ]:
response = openai.ChatCompletion.create(
model="gpt-3.5-turbo-16k",
messages=[
{"role": "system", "content": "You are a helpful assistant that generates questions."},
{"role": "user", "content": transcript_text},
{"role": "user", "content": "Generate 65 quiz questions based on the text with multiple choices."},
]
)

# The assistant's reply
quiz_questions = response['choices'][0]['message']['content']

print("Quiz Questions:")
print(quiz_questions)
